# ECG time series clustering using Self organising maps and Dynamic Time Warping Barycenter Averaging 

In [51]:
# Native libraries
import os
import math
# Essential Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

In [ ]:
def plot_som_series_dba_center(som_x, som_y, win_map):
    fig, axs = plt.subplots(som_x,som_y,figsize=(5,5))
    fig.suptitle('Clusters')
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            if cluster in win_map.keys():
                for series in win_map[cluster]:
                    print(series)
                    axs[cluster].plot(series,c="gray",alpha=0.5) 
                axs[cluster].plot(dtw_barycenter_averaging(np.vstack(win_map[cluster])),c="red") # I changed this part
            cluster_number = x*som_y+y+1
            axs[cluster].set_title(f"Cluster {cluster_number}")

    plt.show()

In [ ]:
# Little handy function to plot series
def plot_som_series_averaged_center(som_x, som_y, win_map):
    fig, axs = plt.subplots(som_x,som_y,figsize=(25,25))
    fig.suptitle('Clusters')
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            if cluster in win_map.keys():
                for series in win_map[cluster]:
                    print(series)
                    axs[cluster].plot(series,c="gray",alpha=0.5) 
                axs[cluster].plot(np.average(np.vstack(win_map[cluster]),axis=0),c="red")
            cluster_number = x*som_y+y+1
            axs[cluster].set_title(f"Cluster {cluster_number}")

    plt.show()

In [ ]:
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(heartbeats))))
# I didn't see its significance but to make the map square,
# I calculated square root of map size which is 
# the square root of the number of series
# for the row and column counts of som

som = MiniSom(10, 10, len(heartbeats[0]), sigma=0.3, learning_rate = 0.1)

som.random_weights_init(heartbeats)
som.train(heartbeats, 1000)

In [ ]:
win_map = som.win_map(heartbeats)
# Returns the mapping of the winner nodes and inputs

plot_som_series_averaged_center(som_x, som_y, win_map)

In [ ]:
win_map = som.win_map(heartbeats)

plot_som_series_dba_center(som_x, som_y, win_map)

In [52]:
# first identify separat heart beats and use them instead whole ecg signal
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [53]:
database_to_frequency = {
    'BIDMC': 125,
    'Fantasia': 250,
    'Capnobase': 300
}

In [54]:
# Define a function to create epochs
def extract_heartbeats(cleaned, peaks, sampling_rate=None): 
    heartbeats = nk.epochs_create(cleaned, 
                                  events=peaks, 
                                  epochs_start=-0.3, 
                                  epochs_end=0.4, 
                                  sampling_rate=sampling_rate)
    heartbeats = nk.epochs_to_df(heartbeats)
    return heartbeats

In [55]:
def ecg_to_heartbeats(row):
    ecg_signal = row['ECG_Signal']
    patient = row['Record']
    fs = database_to_frequency[row['Database']]
    bidmc_heartbeats = []
    try:
        signals, info = nk.ecg_process(ecg_signal, sampling_rate=fs)  
    except Exception as ex:
        signals, info = nk.ecg_process(ecg_signal, sampling_rate=fs, method='pantompkins1985')
    
    finally:   
        rpeaks = info["ECG_R_Peaks"]
        cleaned_ecg = signals["ECG_Clean"]

        heartbeats = extract_heartbeats(cleaned_ecg, peaks=rpeaks, sampling_rate=125)
        heartbeats_pivoted = heartbeats.pivot(index='Time', columns='Label', values='Signal')

        labels = list(heartbeats_pivoted)
        for label in labels:
            bidmc_heartbeats.append(heartbeats_pivoted[label].values.tolist())
    return bidmc_heartbeats

In [56]:
bidmc_df = pd.read_parquet('bidmc_data.parquet')

In [57]:
bidmc_df['heartbeats'] = bidmc_df.apply(ecg_to_heartbeats, axis=1)

In [58]:
fantasia_df = pd.read_parquet('fantasia_data.parquet')
fantasia_df['heartbeats'] = fantasia_df.apply(ecg_to_heartbeats, axis=1)

In [59]:
capnobase_df = pd.read_parquet('capnobase_data.parquet')
capnobase_df['heartbeats'] = capnobase_df.apply(ecg_to_heartbeats, axis=1)

In [60]:
bidmc_df.to_parquet('data/bidmc_heartbeats.parquet')

In [61]:
capnobase_df.to_parquet('data/capnobase_heartbeats.parquet')

In [62]:
fantasia_df.to_parquet('data/fantasia_heartbeats.parquet')

In [ ]:
df = pd.concat([bidmc_df['heartbeats'].explode('heartbeats'), fantasia_df['heartbeats'].explode('heartbeats'), capnobase_df['heartbeats'].explode('heartbeats')]).to_frame('heartbeats')

In [ ]:
df

In [ ]:
df.to_parquet('heartbeats.parquet')

In [ ]:
df = pd.read_parquet('heartbeats.parquet')

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(columns=['index'], inplace=True)

In [ ]:
df['heartbeats_scaled'] = df['heartbeats'].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [ ]:
df

In [ ]:
heartbeats = df['heartbeats'].tolist()

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
# File to store pairwise distances incrementally
distances_file = "pairwise_distances.txt"

num_heartbeats = len(heartbeats)
dtw_matrix = np.zeros((num_heartbeats, num_heartbeats))

# Compute the DTW distances and write to file
with open(distances_file, 'w') as f:
    for i in range(num_heartbeats):
        for j in range(i+1, num_heartbeats):
            distance, _ = fastdtw([heartbeats[i]], [heartbeats[j]], dist=euclidean)
            # Write the distances in the format: i,j,distance
            f.write(f"{i},{j},{distance}\n")

# To populate the matrix from the file:
dtw_matrix = np.zeros((num_heartbeats, num_heartbeats))
with open(distances_file, 'r') as f:
    for line in f:
        i, j, distance = map(float, line.strip().split(","))
        i, j = int(i), int(j)
        dtw_matrix[i, j] = distance
        dtw_matrix[j, i] = distance  # since the matrix is symmetric

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
# Initialize an empty matrix for DTW distances
num_heartbeats = len(heartbeats)
dtw_matrix = np.zeros((num_heartbeats, num_heartbeats))

# Compute the DTW distances
for i in range(num_heartbeats):
    for j in range(i+1, num_heartbeats):
        distance, _ = fastdtw([heartbeats[i]], [heartbeats[j]], dist=euclidean)
        dtw_matrix[i, j] = distance
        dtw_matrix[j, i] = distance  # since the matrix is symmetric

In [ ]:
from joblib import Parallel, delayed

def compute_dtw(i, heartbeats):
    distances = []
    for j in range(i+1, len(heartbeats)):
        distance, _ = fastdtw([heartbeats[i]], [heartbeats[j]], dist=euclidean)
        distances.append((i, j, distance))
    return distances

num_heartbeats = len(heartbeats)
dtw_matrix = np.zeros((num_heartbeats, num_heartbeats))

# Compute the DTW distances in parallel using joblib
results = Parallel(n_jobs=-1)(delayed(compute_dtw)(i, heartbeats) for i in range(num_heartbeats))

# Fill the dtw_matrix with computed distances
for distances in results:
    for i, j, distance in distances:
        dtw_matrix[i, j] = distance
        dtw_matrix[j, i] = distance  # since the matrix is symmetric


In [ ]:
# File to store pairwise distances incrementally
distances_file = "pairwise_distances.txt"

# Compute the DTW distances and write to file
with open(distances_file, 'w') as f:
    for i in range(num_heartbeats):
        for j in range(i+1, num_heartbeats):
            distance, _ = fastdtw(heartbeats[i], heartbeats[j], dist=euclidean)
            # Write the distances in the format: i,j,distance
            f.write(f"{i},{j},{distance}\n")

# To populate the matrix from the file:
dtw_matrix = np.zeros((num_heartbeats, num_heartbeats))
with open(distances_file, 'r') as f:
    for line in f:
        i, j, distance = map(float, line.strip().split(","))
        i, j = int(i), int(j)
        dtw_matrix[i, j] = distance
        dtw_matrix[j, i] = distance  # since the matrix is symmetric

In [ ]:
"""
Much of the code is modified from:
- https://codesachin.wordpress.com/2015/11/28/self-organizing-maps-with-googles-tensorflow/
"""

import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
 

class SOM(nn.Module):
    """
    2-D Self-Organizing Map with Gaussian Neighbourhood function
    and linearly decreasing learning rate.
    """
    def __init__(self, m, n, dim, niter, alpha=None, sigma=None):
        super(SOM, self).__init__()
        self.m = m
        self.n = n
        self.dim = dim
        self.niter = niter
        if alpha is None:
            self.alpha = 0.3
        else:
            self.alpha = float(alpha)
        if sigma is None:
            self.sigma = max(m, n) / 2.0
        else:
            self.sigma = float(sigma)

        self.weights = torch.randn(m*n, dim)
        self.locations = torch.LongTensor(np.array(list(self.neuron_locations())))
        self.pdist = nn.PairwiseDistance(p=2)

    def get_weights(self):
        return self.weights

    def get_locations(self):
        return self.locations

    def neuron_locations(self):
        for i in range(self.m):
            for j in range(self.n):
                yield np.array([i, j])

    def map_vects(self, input_vects):
        to_return = []
        for vect in input_vects:
            min_index = min([i for i in range(len(self.weights))],
                            key=lambda x: np.linalg.norm(vect-self.weights[x]))
            to_return.append(self.locations[min_index])

        return to_return

    def forward(self, x, it):
        dists = self.pdist(torch.stack([x for i in range(self.m*self.n)]), self.weights)
        _, bmu_index = torch.min(dists, 0)
        bmu_loc = self.locations[bmu_index,:]
        bmu_loc = bmu_loc.squeeze()
        
        learning_rate_op = 1.0 - it/self.niter
        alpha_op = self.alpha * learning_rate_op
        sigma_op = self.sigma * learning_rate_op

        bmu_distance_squares = torch.sum(torch.pow(self.locations.float() - torch.stack([bmu_loc for i in range(self.m*self.n)]).float(), 2), 1)
        
        neighbourhood_func = torch.exp(torch.neg(torch.div(bmu_distance_squares, sigma_op**2)))
        
        learning_rate_op = alpha_op * neighbourhood_func

        learning_rate_multiplier = torch.stack([learning_rate_op[i:i+1].repeat(self.dim) for i in range(self.m*self.n)])
        delta = torch.mul(learning_rate_multiplier, (torch.stack([x for i in range(self.m*self.n)]) - self.weights))                                         
        new_weights = torch.add(self.weights, delta)
        self.weights = new_weights

In [ ]:
data = list()
for i in range(len(heartbeats)):
    data.append(torch.FloatTensor(heartbeats[i]))

In [ ]:
m = 10
n = 10
#Train a 20x30 SOM with 100 iterations
n_iter = 100
som = SOM(m, n, 88, n_iter)
for iter_no in range(n_iter):
    #Train with each vector one by one
    for i in range(len(data)):
        som(data[i], iter_no)

In [ ]:
#Store a centroid grid for easy retrieval later on
centroid_grid = [[] for i in range(m)]
weights = som.get_weights()
locations = som.get_locations()
for i, loc in enumerate(locations):
    centroid_grid[loc[0]].append(weights[i].numpy())
 
#Get output grid
image_grid = centroid_grid

#Map colours to their closest neurons
mapped = som.map_vects(torch.Tensor(colors))

#Plot
plt.imshow(image_grid)
plt.title('Color SOM')
for i, m in enumerate(mapped):
    plt.text(m[1], m[0], color_names[i], ha='center', va='center',
             bbox=dict(facecolor='white', alpha=0.5, lw=0))
plt.show()